## Process DSF data

We want to compute the `Dynamical Structure Factor`, which is given by

$$ S_{\vec{q}}(w) = \sum_{n=1}^{nev} \left| \left\langle \Psi_n^X \left| \hat{S}_{\vec{q}}^{\alpha} \right| \Psi_n^0 \right\rangle \right| ^2 \delta(\omega - (E_n^X - E_0)) $$

$$ S_{\vec{q}}(w) = \sum_{n=1}^{nev} \left| \left\langle \Psi_n^X \left| \hat{S}_{\vec{q}}^{\alpha} \right| \Psi_n^0 \right\rangle \right| ^2 \frac{1}{\pi} \frac{\epsilon^2}{(\omega - (E_n^X - E_0))^2 + \epsilon^2} $$

From the code we obtain a file with two columns: 
1. the first one, has the values of $ \Delta_n = E_n^X -E_0 $
2. the second one, has the values of the coefficient $ \left\langle \Psi_n^X \left| \hat{S}_{\vec{q}}^{\alpha} \right| \Psi_n^0 \right\rangle $

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from mpl_toolkits.mplot3d import axes3d
from matplotlib.collections import PolyCollection
%matplotlib notebook

### Parameters

In [ ]:
dim      = 2            # Dimension of the lattice
nspins   = 32           # Number of spins in the system

if dim == 1:
    Lx   = 1            # Number of spins in the system in x direction
    Ly   = nspins       # Number of spins in the system in y direction
else:
    Lx   = 4
    Ly   = 8
    
rep      = 1            # Disorder realizations
nev      = 150          # Number of excited states used

#q_idx    = [i for i in range(24)] #Indices of q to take
q_idx    = [0,1,2,3,4,8,9,10,11,12,16,17,18,19,20]
q_idxX   = [i//Ly for i in q_idx]
q_idxY   = [i%Ly for i in q_idx]
print("q:", q_idx)
print("qx:", q_idxX)
print("qy:", q_idxY)
nQ       = len(q_idx)   # Number of q values to analyse

J1       = 1.0          # Nearest neighbours coupling
D1       = 1.0          # Nearest neighbours anisotropy
J2       = 1.0          # Next-nearest neighbours coupling
D2       = 1.0          # Next-nearest neighbours anisotropy

nW       = 5001         # Number of frequency values to plot
w_min    = 0            # Minimum value of frequency to plot
w_max    = 5            # Maximum value of frequency to plot
epsilon  = 1.0/nspins   # Constant to compute the Dirac delta function

path2INdata  = 
path2OUTdata = 

### Prepare data

#### 1. Read data from files

In [ ]:
# DeltaEn and factorN have the values of (En-E0) and the preceeding factor of the DSF
DeltaEn = np.empty((nev*nQ,rep))
factorN = np.empty((nev*nQ,rep))

# Loop over disorder realizations
for dis_rep in range(rep):
    
    # Loop over excited states
    with open(path2INdata + 'dsf_d' + str(dis_rep) + '_' + str(nspins) + '_' + str(nev) + '.dat') as INfile:
        array = np.genfromtxt(INfile)
        tmp_DeltaEn, tmp_factorN = array.T
            
    
    DeltaEn[:,dis_rep] = tmp_DeltaEn
    factorN[:,dis_rep] = tmp_factorN

#### 2. Create the frequency-dependent variables

In [ ]:
    delta_w  = (w_max - w_min)/(nW - 1)
    w  = np.linspace(w_min,w_max,nW)
    Sw = np.zeros((nW,nQ))

    # Funtion to compute the Dirac delta
    pi     = np.arccos(-1)
    inv_pi = 1.0/pi

    def DiracDelta(w,deltaEn,e2):
        return inv_pi*e2/(e2 + (w-deltaEn)*(w-deltaEn))

    # Loop over q values
    e2 = epsilon*epsilon

    # Loop over disorder realizations
    for dis_rep in range(rep):  

        # Loop over q values
        tmp_Sw = np.zeros((rep,nW,nQ))
        for wi in range(nW):

            for q in range(nQ):
                start_off = q*nev+1
                end_off   = (q+1)*nev
                mid_array = factorN[start_off:end_off,dis_rep] * DiracDelta(w[wi],DeltaEn[start_off:end_off,dis_rep],e2)
                tmp_Sw[dis_rep,wi,q] = np.sum(mid_array)

    # Compute average over disorder realizations
    Sw = np.sum(tmp_Sw, axis=0)
    Sw *= 2*pi
    Sw /= rep

#### 3. Write in file

In [ ]:
header = "        w                 Sw"
np.savetxt(path2OUTdata + 'Sq.csv', np.c_[w,Sw], header=header, fmt='%16.13e', delimiter=' ',)      

### Plot

#### 1. Plot for every q separately

In [ ]:
# Plot parameters
subsX = nQ//4
subsY = 4
if subsX*subsY != nQ:
    subsX += 1

fig, ax = plt.subplots(subsX, subsY, gridspec_kw={'wspace':0.5, 'hspace': 0.5}, figsize=(20,20))
fig.suptitle("Dynamical Structure Factor for different q values", fontsize=30)


for i in range(subsX):
    for j in range(subsY):
        qij = i*subsY + j
        #print(qij,q_idx[qij])
        if qij >= nQ:
            break
        ax[i,j].set_title('q: ' + str(q_idx[qij]) + '--> q = (' + str(q_idxX[qij]) + ',' + str(q_idxY[qij]) + ')')
        ax[i,j].set_xlabel('w')
        ax[i,j].set_ylabel('S(w)')
        ax[i,j].ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        ax[i,j].grid()
        ax[i,j].plot(w,Sw[:,qij])

In [ ]:
if dim == 2 and nspins == 32:
    fig3, ax3 = plt.subplots(1,2,gridspec_kw={'wspace':0.5, 'hspace': 0.5}, figsize=(20,5))

    ax3[0].set_title('S(w) for q=20 --> LARGER peak')
    ax3[0].set_xlabel('w')
    ax3[0].set_ylabel('S(w)')
    ax3[0].grid(which='both')
    ax3[0].plot(w, Sw[:,14])

In [ ]:
if dim == 2 and nspins == 16:
    fig3, ax3 = plt.subplots(1,2,gridspec_kw={'wspace':0.5, 'hspace': 0.5}, figsize=(20,5))

    ax3[0].set_title('S(w) for q=10 --> LARGER peak')
    ax3[0].set_xlabel('w')
    ax3[0].set_ylabel('S(w)')
    ax3[0].grid()
    ax3[0].plot(w, Sw[:,10])

    ax3[1].set_title('S(w) for q=10 --> SMALLER peak')
    ax3[1].set_xlabel('w')
    ax3[1].set_ylabel('S(w)')
    ax3[1].grid()
    ax3[1].plot(w[4000:5000],Sw[4000:5000,10])

In [ ]:
if dim == 2 and nspins == 16:
    # For (pi,pi) --> q=10 check the ratio between peaks
    max1 = np.argmax(Sw,axis=0)[10]
    max2 = np.argpartition(Sw[4000:5000,10],-1)[-1:]
    
    ratio = Sw[max1,10]/Sw[4000+max2[0],10]
    print('Ratio between larget peak and smaller one:', ratio)
    print('Approx ratio from literature (Chen,Schuttler PhysRevB1989):', 15.4/0.174)
    #print(max1, max2)
    #print(Sw[579,10])
    #print(Sw[4000+max2[0],10])

#### 2. Plot all q values together

In [ ]:
# Maximum value over all Sq(w)
max = 0
for qi in range(nQ):
    tmp_max = Sw[:,qi].max()
    if tmp_max >= max:
        max = tmp_max

# q values for plotting
q = np.linspace(0,nQ-1,nQ)

# Prepare data for 3D plot
verts = []
for qi in range(nQ):
    verts.append(list(zip(w,Sw[:,qi])))

# Set colors
fc = []
for qi in range(nQ):
    fc.append(((qi%10)*0.1+0.09, (qi%10)*0.1+0.1, (qi%10)*0.1+0.08))

# Define the 3D plot
poly = PolyCollection(verts, facecolors = fc)
poly.set_alpha(0.7)

fig2 = plt.figure(figsize=(20,20))
ax2  = fig2.add_subplot(111, projection='3d')
ax2.add_collection3d(poly, zs=q, zdir='y')

# Plot details
ax2.set_title('Dynamical Structure Factor', fontsize=30)
ax2.ticklabel_format(axis='z', style='sci', scilimits=(0,0))

ax2.set_xlim3d(w.min(), w.max())
ax2.set_xlabel('w')
ax2.set_ylim3d(q.min(), q.max())
ax2.set_ylabel('q')
ax2.set_yticks(np.arange(0,nQ,1))
ax2.set_zlim3d(Sw[:,2].min(), max)
ax2.set_zlabel('Sq(w)')

plt.show()

### Checks

##### 1. Sum rule: $ \sum_{\vec{q}} \int \frac{d\omega}{2\pi} S^{\gamma}_{\vec{q}}(\omega) = \frac{1}{4} 
\rightarrow 
\frac{\Delta\omega}{2\pi} \sum_{\vec{q}} \sum_i^{nW} S^{\gamma}_{\vec{q}}(\omega_i) = \frac{1}{4}$

In [ ]:
sum_rule = np.sum(Sw) * delta_w / (2*pi)
e_msg = "Sum rule does not match: sum_rule = " + str(sum_rule)
assert abs(sum_rule - 0.25)<0.01, e_msg
print('Sum rule: ', sum_rule)

#### 2. Dispersion relationship (ONLY FOR 1D)

In [ ]:
if dim == 1:
    # Get the positions of the highest peak for each q
    disp_w = np.argmax(Sw,axis=0)

    disp_fig, disp_ax = plt.subplots()

    # Plot dispersion along y axis
    #x0 = 2
    #q_y = np.zeros((Ly))
    #for i in range(Ly):
    #q_y[i] = i + x0 * Lx
    #qy = q_y.astype(int)
    #print(qy)

    disp_ax.set_title('Dispersion w(q=2pi/L * n)')
    disp_ax.set_ylabel('w(q)')
    disp_ax.set_xlabel('q')
    disp_ax.grid()
    disp_ax.plot(q/Ly*2*pi, disp_w*delta_w)
    disp_ax.plot(q/Ly*2*pi,pi/2*abs(np.sin(q/Ly*2*pi)))

#### 2. Dispersion relationship (ONLY FOR 2D)

In [ ]:
if dim == 2:
    # Get the positions of the highest peak for each q
    disp_w = np.argmax(Sw,axis=0)

    disp_fig, disp_ax = plt.subplots(1,2,gridspec_kw={'wspace':0.5, 'hspace': 0.5}, figsize=(20,5))

    # Plot dispersion along y axis
    x0 = 2
    q_y = np.zeros((Ly))
    for i in range(Ly):
        
        q_y[i] = i + x0 * Lx
        qy = q_y.astype(int)
    print('Values of q printing (along Y):', qy)

    disp_ax[0].set_title('Dispersion w(q=2pi/L * n) along Y')
    disp_ax[0].set_ylabel('w(q)')
    disp_ax[0].set_xlabel('q')
    disp_ax[0].grid()
    disp_ax[0].plot(q[0:4]/Ly*2*pi, disp_w[qy]*delta_w)
    disp_ax[0].plot(q[0:4]/Ly*2*pi,np.sqrt(4-(np.cos(x0/Lx*2*pi)+np.cos(q[0:4]/Ly*2*pi))**2))
    
    # Plot dispersion along x axis
    y0 = 1
    q_x = np.zeros((Lx))
    for i in range(Lx):
        q_x[i] = Ly * i + y0
        qx = q_x.astype(int)
    print('Values of q printing (along X):', qx)

    disp_ax[1].set_title('Dispersion w(q=2pi/L * n) along X')
    disp_ax[1].set_ylabel('w(q)')
    disp_ax[1].set_xlabel('q')
    disp_ax[1].grid()
    disp_ax[1].plot(q[0:4]/Lx*2*pi, disp_w[qx]*delta_w)
    disp_ax[1].plot(q[0:4]/Lx*2*pi,np.sqrt(4-(np.cos(q[0:4]/Lx*2*pi)+np.cos(y0/Ly*2*pi))**2))